## Task #8

In [145]:
import numpy as np
import pandas as pd
from scipy.stats import t, levene, bartlett, fligner

data = pd.read_excel("Данные для тестового задания.xlsx", sheet_name = "Данные АБ тестов")

In [146]:
exp_1_control = data[(data.experiment_num == 1) & (data.experiment_group == 'control')]['revenue']
exp_1_test = data[(data.experiment_num == 1) & (data.experiment_group == 'test')]['revenue']
exp_2_control = data[(data.experiment_num == 2) & (data.experiment_group == 'control')]['revenue']
exp_2_test = data[(data.experiment_num == 2) & (data.experiment_group == 'test')]['revenue']
exp_3_control = data[(data.experiment_num == 3) & (data.experiment_group == 'control')]['revenue']
exp_3_test = data[(data.experiment_num == 3) & (data.experiment_group == 'test')]['revenue']

In [306]:
user_id_distinct = data.groupby('user_id').size().reset_index(name = 'counts')
user_id_distinct[user_id_distinct.counts != 3]

,user_id,counts


Выходит что каждый юзер поучаствовал в трех экспериментах.

### Experiment 1

In [316]:
# У нас достаточный размер выборок, но неизвестно стандартное отклонение ген. совокупности, значит используем t-test
# exp_1_test среднее меньше чем среднее значение exp_1_control

# H0: ARPU экспериментальной и контрольной групп равны
# H1: ARPU экспериментальной и контрольной групп не равны

# Равность дисперсий
levene_stat, levene_p = levene(exp_1_test, exp_1_control, center = 'mean')
bartlett_stat, bartlett_p = bartlett(exp_1_test, exp_1_control)
fligner_stat, fligner_p = fligner(exp_1_test, exp_1_control)

results_var_eq = pd.DataFrame({
    "Test": ["Levene", "Bartlett", "Fligner-Killeen"],
    "Statistic": [levene_stat, bartlett_stat, fligner_stat],
    "p-value": [levene_p, bartlett_p, fligner_p],
    "Equal Variances? (α = 0.05)": ["Yes" if p > 0.05 else "No" for p in [levene_p, bartlett_p, fligner_p]]
})
results_var_eq

,Test,Statistic,p-value,Equal Variances? (α = 0.05)
0,Levene,0.372572,0.541753,Yes
1,Bartlett,0.707828,0.400166,Yes
2,Fligner-Killeen,0.531582,0.465942,Yes


In [225]:
# уровень значимости = 0.05
def standard_error_two_samples(sample1, sample2):
    n1, n2 = len(sample1), len(sample2)
    var1, var2 = sample1.var(ddof = 1), sample2.var(ddof = 1)

    levene_stat, levene_p = levene(sample1, sample2, center='mean')
    equal_var = True if levene_p > 0.05 else False
    
    if equal_var:
        sp = np.sqrt(((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2))
        se = sp * np.sqrt(1/n1 + 1/n2)
        dof = n1 + n2 - 2
    else:
        se = np.sqrt(var1/n1 + var2/n2)
        dof = (var1/n1 + var2/n2)**2 / ((var1/n1)**2/(n1-1) + (var2/n2)**2/(n2-1))
    return se, dof

mean_diff_exp_1 = exp_1_test.mean() - exp_1_control.mean()
std_err_exp_1, dof_exp_1 = standard_error_two_samples(exp_1_test, exp_1_control)

t_stat_exp_1 = mean_diff_exp_1 / std_err_exp_1
p_val_exp1 = 2 *(1 - t.cdf(abs(t_stat_exp_1), dof_exp_1))
p_val_exp1

0.6887842117790162

Из-за того что *p_value > 0.05* результат статистически не значимый и мы не можем утверждать что ***ARPU*** экспериментальной и контрольной не равны.

In [228]:
# Дальше используем готовую функцию
from scipy.stats import ttest_ind

equal_var = True if levene_p > 0.05 else False

t_stat_exp_1, p_val_exp1 = ttest_ind(exp_1_test, exp_1_control, equal_var = equal_var)
p_val_exp1

0.688784211779017

In [318]:
# Находим доверительный интервал α = 0.05 двусторонний для реальной разницы ARPU
alpha = 0.05
t_critical = t.ppf(1 - alpha/2, dof_exp_1)

lower_CI_exp1 = mean_diff_exp_1 - t_critical * std_err_exp_1
upper_CI_exp1 = mean_diff_exp_1 + t_critical * std_err_exp_1

(lower_CI_exp1, upper_CI_exp1)

(-334.5670447711731, 221.12578133031286)

Из-за того что *p_value/2 > 0.05*, то несмотря на то какую одностороннюю альтернативную гипотезу мы бы ни тестили (experimental_mean > control_mean OR experimental_mean < control_mean), нулевую гипотезу мы отклонить все равно не сможем. Делаем вывод, что у нас не достаточно доказательств чтоб утверждать что у нас вообще есть какое-либо отличие в ARPU между экспериментальной и контрольной выборками в первом эксперименте.

### Experiment 2

In [312]:
# У нас достаточный размер выборок, но неизвестно стандартное отклонение ген. совокупности, значит используем t-test
# exp_2_test среднее меньше чем среднее значение exp_2_control

# H0: ARPU экспериментальной и контрольной групп равны
# H1: ARPU экспериментальной и контрольной групп не равны

# Равность дисперсий
levene_stat, levene_p = levene(exp_2_test, exp_2_control, center = 'mean')
bartlett_stat, bartlett_p = bartlett(exp_2_test, exp_2_control)
fligner_stat, fligner_p = fligner(exp_2_test, exp_2_control)

results_var_eq = pd.DataFrame({
    "Test": ["Levene", "Bartlett", "Fligner-Killeen"],
    "Statistic": [levene_stat, bartlett_stat, fligner_stat],
    "p-value": [levene_p, bartlett_p, fligner_p],
    "Equal Variances? (α = 0.05)": ["Yes" if p > 0.05 else "No" for p in [levene_p, bartlett_p, fligner_p]]
})
results_var_eq

,Test,Statistic,p-value,Equal Variances? (α = 0.05)
0,Levene,28.426511,1.218390e-07,No
1,Bartlett,233.052045,1.287541e-52,No
2,Fligner-Killeen,60.995157,5.721539e-15,No


In [236]:
# уровень значимости = 0.05

equal_var = True if levene_p > 0.05 else False

t_stat_exp_2, p_val_exp2 = ttest_ind(exp_2_test, exp_2_control, equal_var = equal_var)
p_val_exp2

0.0011282266247294027

Из-за того что *p_value < 0.05* результат статистически значимый и мы можем отклонить нулевую гипотезу и утвердить что ***ARPU*** экспериментальной и контрольной групп не равны.

In [239]:
# H0: ARPU экспериментальной и контрольной групп равны
# H1: ARPU экспериментальной < ARPU контрольной

p_val_exp2 / 2

0.0005641133123647014

Из-за того что *p_value/2 < 0.05*, мы можем отклонить нулевую гипотезу. Делаем вывод, что у нас есть достаточно доказательств чтоб утверждать что у нас ***ARPU*** экспериментальной группы меньше контрольной.

In [320]:
# Находим доверительный интервал α = 0.05 односторонний для реальной разницы ARPU
alpha = 0.05
std_err_exp_2, dof_exp_2 = standard_error_two_samples(exp_2_test, exp_2_control)
t_critical = t.ppf(1 - alpha, dof_exp_2)
mean_diff_exp_2 = exp_2_test.mean() - exp_2_control.mean()

lower_CI_exp2 = - np.inf 
upper_CI_exp2 = mean_diff_exp_2 + t_critical * std_err_exp_2

(lower_CI_exp2, upper_CI_exp2)

(-inf, -184.52210368217484)

### Experiment 3

In [314]:
# У нас достаточный размер выборок, но неизвестно стандартное отклонение ген. совокупности, значит используем t-test
# exp_3_test среднее больше чем среднее значение exp_2_control

# H0: ARPU экспериментальной и контрольной групп равны
# H1: ARPU экспериментальной и контрольной групп не равны

# Равность дисперсий
levene_stat, levene_p = levene(exp_3_test, exp_3_control, center = 'mean')
bartlett_stat, bartlett_p = bartlett(exp_3_test, exp_3_control)
fligner_stat, fligner_p = fligner(exp_3_test, exp_3_control)

results_var_eq = pd.DataFrame({
    "Test": ["Levene", "Bartlett", "Fligner-Killeen"],
    "Statistic": [levene_stat, bartlett_stat, fligner_stat],
    "p-value": [levene_p, bartlett_p, fligner_p],
    "Equal Variances? (α = 0.05)": ["Yes" if p > 0.05 else "No" for p in [levene_p, bartlett_p, fligner_p]]
})
results_var_eq

,Test,Statistic,p-value,Equal Variances? (α = 0.05)
0,Levene,6.159484,1.324414e-02,No
1,Bartlett,63.633702,1.498436e-15,No
2,Fligner-Killeen,32.262068,1.347178e-08,No


In [266]:
# уровень значимости = 0.05

equal_var = True if levene_p > 0.05 else False

t_stat_exp_3, p_val_exp3 = ttest_ind(exp_3_test, exp_3_control, equal_var = equal_var)
p_val_exp3

0.06031548002841902

Из-за того что *p_value > 0.05* результат статистически не значимый и мы не можем отклонить нулевую гипотезу и не можем утвердить что ***ARPU*** экспериментальной и контрольной групп не равны.

In [269]:
# H0: ARPU экспериментальной и контрольной групп равны
# H1: ARPU экспериментальной > ARPU контрольной

p_val_exp3 / 2

0.03015774001420951

Из-за того что *p_value/2 < 0.05*, мы можем отклонить нулевую гипотезу. Делаем вывод, что у нас есть достаточно доказательств чтоб утверждать что у нас ***ARPU*** экспериментальной группы больше контрольной.

In [322]:
# Находим доверительный интервал α = 0.05 односторонний для реальной разницы ARPU
alpha = 0.05
std_err_exp_3, dof_exp_3 = standard_error_two_samples(exp_3_test, exp_3_control)
t_critical = t.ppf(1 - alpha, dof_exp_3) 
mean_diff_exp_3 = exp_3_test.mean() - exp_3_control.mean()

lower_CI_exp3 = mean_diff_exp_3 - t_critical * std_err_exp_3
upper_CI_exp3 = np.inf

(lower_CI_exp3, upper_CI_exp3)

(41.792469588239555, inf)

### Результат

In [328]:
print("Сравниваем Experimental VS Control (то есть от Experimental отнимаем Control в анализах.)")

results = pd.DataFrame({
    "Experiments": ["Experiment 1", "Experiment 2", "Experiment 3"],
    "Two OR one-sided": ["Two-sided", "One-sided less", "One-sided more"],
    "t-statistic": [t_stat_exp_1, t_stat_exp_2, t_stat_exp_3],
    "p-value": [p_val_exp1, p_val_exp2/2, p_val_exp2/2],
    "CI": [(lower_CI_exp1, upper_CI_exp1), (lower_CI_exp2, upper_CI_exp2), (lower_CI_exp3, upper_CI_exp3)]
})
results

Сравниваем Experimental VS Control (то есть от Experimental отнимаем Control в анализах.)


,Experiments,Two OR one-sided,t-statistic,p-value,CI
0,Experiment 1,Two-sided,-0.400629,0.688784,"(-334.5670447711731, 221.12578133031286)"
1,Experiment 2,One-sided less,-3.270721,0.000564,"(-inf, -184.52210368217484)"
2,Experiment 3,One-sided more,1.880979,0.000564,"(41.792469588239555, inf)"


## Task #18

In [30]:
import numpy as np
from statsmodels.stats.proportion import proportions_ztest, confint_proportions_2indep

paid = np.array([1003, 1099])
samples = np.array([100047501, 100001055])

# H0: конверсии двух выборок одинаковы
# H1: конверсии двух выборок не одинаковы

z_stat_two_sided, p_val_two_sided = proportions_ztest(paid, samples, alternative = 'two-sided')

# Print the results
print(f"z-statistic two-sided: {z_stat_two_sided:.4f}")
print(f"p-value two-sided: {p_val_two_sided:.4f}")

z-statistic two-sided: -2.1046
p-value two-sided: 0.0353


In [36]:
# H0: конверсии двух выборок одинаковы
# H1: конверсия контрольной группы меньше чем конверсия 

z_stat_one_sided, p_val_one_sided = proportions_ztest(paid, samples, alternative = 'smaller')

# Print the results
print(f"z-statistic one-sided smaller: {z_stat_one_sided:.4f}")
print(f"p-value one-sided smaller: {p_val_one_sided:.4f}")

effect_size_h = 2 * (np.arcsin(np.sqrt(paid[0]/samples[0])) - np.arcsin(np.sqrt(paid[1]/samples[1])))
print(f"effect size for Two Proportions Z test: {effect_size_h:.4f}")

z-statistic one-sided smaller: -2.1046
p-value one-sided smaller: 0.0177
effect size for Two Proportions Z test: -0.0003
